In [9]:
import pandas as pd
from pymongo import MongoClient

In [10]:
# Set connection
client = MongoClient("mongodb://localhost:27017/")
db = client["bank"]
balances_collection = db["balances"]

# Load csv
accounts = pd.read_csv("sample/accounts.csv")
transactions = pd.read_csv("sample/transactions.csv", parse_dates=["tran_timestamp"])

In [11]:
print(transactions["tran_timestamp"].max())  # Should be 2018-12-21

2018-12-21 00:00:00+00:00


In [12]:
# Create root
balances = {}

OPEN_DATE_REFERENCE = "2017-01-01"  # Define the reference date

# Take users data from account.csv
for _, row in accounts.iterrows():
    acc_id = row["acct_id"] # User id
    open_date = OPEN_DATE_REFERENCE if row["open_dt"] == 0 else row["open_dt"]
    initial_balance = round(float(row["initial_deposit"]), 2)

    balances[acc_id] = [{"date": open_date,
                         "balance": initial_balance
                        }]
    
    if acc_id == 0: # Check
        print("first user must be 0 => ",str(acc_id))
        print("user 0 initial deposit must be 96646.86 => ", initial_balance)

first user must be 0 =>  0
user 0 initial deposit must be 96646.86 =>  96646.86


In [13]:
print(len(transactions))

121457


In [14]:
# Apply transactions to users' balances
transactions.sort_values("tran_timestamp", inplace=True)

for i, (_, row) in enumerate(transactions.iterrows()):
    date = row["tran_timestamp"].strftime("%Y-%m-%d")
    orig_acct = str(row["orig_acct"])
    bene_acct = str(row["bene_acct"])

    if i == 0:
        print("first transaction must be from 982 to 23 => ", orig_acct, bene_acct)
    if i == 121456: # check that read till the last transaction
        print("last transaction must be from 979 to 10 => ", orig_acct, bene_acct)

    amount = row["base_amt"]
    # aml_flag = row["is_sar"]

    if orig_acct in balances:
        new_balance = balances[orig_acct][-1]["balance"] - amount
        balances[orig_acct].append({"date": date, "balance": new_balance})
    
    if bene_acct in balances:
        new_balance = balances[bene_acct][-1]["balance"] + amount
        balances[bene_acct].append({"date": date, "balance": new_balance})

first transaction must be from 982 to 23 =>  982 23
last transaction must be from 979 to 10 =>  979 10


In [15]:
balances

{0: [{'date': '2017-01-01', 'balance': 96646.86}],
 1: [{'date': '2017-01-01', 'balance': 80707.36}],
 2: [{'date': '2017-01-01', 'balance': 52101.1}],
 3: [{'date': '2017-01-01', 'balance': 90324.4}],
 4: [{'date': '2017-01-01', 'balance': 91267.86}],
 5: [{'date': '2017-01-01', 'balance': 77717.44}],
 6: [{'date': '2017-01-01', 'balance': 69099.23}],
 7: [{'date': '2017-01-01', 'balance': 97479.13}],
 8: [{'date': '2017-01-01', 'balance': 98752.44}],
 9: [{'date': '2017-01-01', 'balance': 88834.29}],
 10: [{'date': '2017-01-01', 'balance': 92168.76}],
 11: [{'date': '2017-01-01', 'balance': 81167.91}],
 12: [{'date': '2017-01-01', 'balance': 73205.69}],
 13: [{'date': '2017-01-01', 'balance': 98213.62}],
 14: [{'date': '2017-01-01', 'balance': 55914.61}],
 15: [{'date': '2017-01-01', 'balance': 55743.03}],
 16: [{'date': '2017-01-01', 'balance': 50963.87}],
 17: [{'date': '2017-01-01', 'balance': 93010.16}],
 18: [{'date': '2017-01-01', 'balance': 92937.23}],
 19: [{'date': '2017-01-

In [16]:
# Send the data to DB
for date in sorted(set(transactions["tran_timestamp"].dt.strftime("%Y-%m-%d"))):
    daily_balances = {
        "date": date,
        "balances": {str(user): {"balance": b[-1]["balance"]} for user, b in balances.items()}
    }
    balances_collection.insert_one(daily_balances)

print("All balances are now updated with transactions from 2017-01-01 to 2018-12-21")

All balances are now updated with transactions
